Please follow instructions first:  
<span style="color:red">***INSTRUCTIONS:***</span>   
<span style="color:red">**(1)&nbsp;General activation: on the main toolbar click "Cell" > "Run all".**</span>  
<span style="color:red">**(2)&nbsp;Widgets activation: In order to use one of them first click the "Reset/Start" button to activate it (The second one is already set).**</span>  
<span style="color:red">**(*)&nbsp;Press "View/Hide Code" button in order to view / hide the raw code.**</span>

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="View/Hide Code"></form>''')

# Nonideal Relay  
## (I copied most of it from wikipedia, just a temporary text)

The relay hysteron is the fundamental building block of the Preisach model. It is described as a two-valued operator denoted by $\alpha$ and $\beta$. $\alpha$  defines the "switch-off" threshold, and $\beta$  defines the "switch-on" threshold.  

Graphically, if u is less than $\alpha$ , the output is "low" or "off." As we increase u, the output remains low until u reaches $\beta$ — at which point the output switches "on." Further increasing u has no change. Decreasing u, output does not go low until u reaches $\alpha$  again. It is apparent that the relay operator takes the path of a loop, and its next state depends on its past state.  
The Preisach model consists of many relay hysterons connected in parallel, given weights, and summed. In the widget 3 equal-weighted relays are connected in parallel and sum output is presented on the right. Each relay can be plotted on a so-called Limited Plane with its $\alpha$ $\beta$ values

In [2]:
%matplotlib widget
# %matplotlib notebook
import ipywidgets as widgets
from ipywidgets import Layout
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from matplotlib.gridspec import GridSpec
import random
from palettable.colorbrewer.sequential import Greys_9



class app(object):
    def __init__(self):
        self.hys_per_side = 101
        self.beta_values = np.around(np.linspace(0, 1, self.hys_per_side),2) # beta values
        self.alpha_values = np.around(np.linspace(1,0, self.hys_per_side),2) # alpha values
        self.beta_grid, self.alpha_grid = np.meshgrid(self.beta_values, self.alpha_values)
        self.preisach_triangle = np.where(self.alpha_grid>self.beta_grid, 0, np.nan)
        
        
        self.out = widgets.Output()
        self.input_u = widgets.FloatSlider(
                            value=0, 
                            min=0, max=1, step=0.01,
                            description='input $u$',
                            continuous_update=True,
                            layout=Layout(width='50%')
                       )
        self.input_u.observe(self.update_app, 'value')
        self.reset_button = widgets.Button(
                                description='Reset / Start',
                                icon="trash",
                                style={'font_weight': 'bold', 'button_color': 'yellow'}
                            )
        self.reset_button.on_click(self.on_button_clicked)# if you click it will activate the function
        self.on_button_clicked(1) # We force a click to reset all the plots
    def draw_arrow(self,ax, start, end):
        ax.annotate('', xy=end, xytext=start, xycoords='data', textcoords='data',
                      arrowprops=dict(headwidth=4.0, headlength=4.0, width=0.2,
                                      facecolor = "black", linewidth = 0.5),zorder=0)
    def on_button_clicked(self, b): # its just the jupiter format for clicking
        plt.clf()
        self.fig = plt.figure(1, figsize=(10,5))
        # self.fig.canvas.layout.width = '1000px'
        gs1 = GridSpec(3, 1, width_ratios=[1],
                             height_ratios=[1, 1, 1],
                             left=0.03, right=0.36,
                             bottom=0.03, top=1.00,
                             wspace=0.6, hspace=0.2)
        gs2 = GridSpec(2, 1, width_ratios=[1],
                             height_ratios=[1, 1],
                             left=0.65, right=0.95,
                             bottom=0.03, top=0.92,
                             wspace=0.6, hspace=0.5)
        self.ax1 = plt.subplot(gs1[0, 0])
        self.ax2 = plt.subplot(gs1[1, 0])
        self.ax3 = plt.subplot(gs1[2, 0])
        self.ax4 = plt.subplot(gs2[0, 0])
        self.ax5 = plt.subplot(gs2[1, 0])
        # x is the same as the input u, but it's a static array, used for plotting
        self.x = np.arange(0, 1, 0.005)
        
        # Setting all 3 hysterons
        self.reset_AB() # randomly draw alpha and beta for the hysterons, then plot
        self.plot_hysteron(self.ax1, self.A1, self.B1, 1)
        self.plot_hysteron(self.ax2, self.A2, self.B2, 2)
        self.plot_hysteron(self.ax3, self.A3, self.B3, 3)        
        
        # Hysteresis plot
        self.ax4.axis([-0.05, 1.2, -0.3, 3.8])
        self.draw_arrow(self.ax4, (-0.035, -0.2),(-0.035,3.4))
        self.ax4.annotate('Output', xy=(-0.035,3.7))
        self.draw_arrow(self.ax4, (-0.035, -0.2),(1.2, -0.2))
        self.ax4.text(1.2, 0.1, "$u$", ha="right")
        self.ax4.spines['top'].set_visible(False)
        self.ax4.spines['right'].set_visible(False)
        self.ax4.spines['bottom'].set_visible(False)
        self.ax4.spines['left'].set_visible(False)
        self.ax4.set_xticks([])
        self.ax4.set_yticks([])
        self.ax4.tick_params(axis=u'both', which=u'both',length=0)

        # Preisach triangle plot
        self.ax5.axis([0.0, 1.2, -0.05, 1.1])
        self.ax5.scatter([self.B1, self.B2, self.B3], [self.A1, self.A2, self.A3], color='tab:red', s=15,zorder=3)
        self.ax5.annotate("1", (self.B1 + 0.03, self.A1 - 0.015),color="tab:red",zorder=3, size=10)
        self.ax5.annotate("2", (self.B2 + 0.03, self.A2 - 0.015),color="tab:red",zorder=3, size=10)
        self.ax5.annotate("3", (self.B3 + 0.03, self.A3 - 0.015),color="tab:red",zorder=3, size=10)
        self.ax5.text(0.5,1.1,"Limit Triangle", ha="center")                  
        self.ax5.text(0.0, 1.22, r"$\alpha$")                                   
        self.ax5.text(1.0, 0.93, r"$u$")          
        self.ax5.text(1.2, 0.05, r"$\beta$")
        self.draw_arrow(self.ax5, (0,0),(1.2, 0))
        self.draw_arrow(self.ax5, (0,0),(0, 1.09))
        self.draw_arrow(self.ax5, (0,0),(1.1, 1.1))
        self.ax5.spines['top'].set_visible(False)
        self.ax5.spines['right'].set_visible(False)
        self.ax5.spines['bottom'].set_visible(False)
        self.ax5.spines['left'].set_visible(False)
        self.ax5.set_xticks([])
        self.ax5.set_yticks([])
        self.ax5.tick_params(axis=u'both', which=u'both',length=0)
        self.ax5.fill_between(self.x, self.x, y2=1.0, color="gray", alpha=0.2,zorder=20)
        self.on_state = self.ax5.fill_between(self.x, self.x, 1.0, color="black",zorder=0)
        
        
        self.input_history = [1.0]
        self.output_history = [3.0]
        self.reset_markers()
        self.input_u.value = 1.0
        
               
    def plot_hysteron(self, ax, A, B, hysteron_number):
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        self.draw_arrow(ax, (0,0),(1, 0))
        self.draw_arrow(ax, (0,0),(0, 1.5))
        ax.text(1.2, 0, "$u$", ha="right")
        ax.plot(np.where(self.x < A , self.x,np.nan), np.where(self.x < A , 0,np.nan), zorder=1,color="black")
        ax.plot(np.where(self.x > A , self.x,np.nan), np.where(self.x > A , 1,np.nan), zorder=1,color="black")
        ax.plot([A,A], [0,1], zorder=1,color="tab:orange")
        ax.plot(np.where(self.x < B , self.x,np.nan), np.where(self.x < B , 0,np.nan), zorder=1,color="black")
        ax.plot(np.where(self.x > B , self.x,np.nan), np.where(self.x > B , 1,np.nan), zorder=1,color="black")
        ax.plot([B,B], [0,1], zorder=1, color="tab:blue")
        ax.plot([A],[0.5],marker="^", color="tab:orange",zorder=0)
        ax.plot([B],[0.5],marker="v", color="tab:blue",zorder=0)
        ax.text(A, -0.25, r"$\alpha$", horizontalalignment="center")
        ax.text(B, -0.25, r"$\beta$", horizontalalignment="center")
        ax.axis([0, 1, -0.2, 1.5])
        ax.text(0.5,1.3,"hysteron {}".format(hysteron_number), ha="center")

    def reset_AB(self):
        """
        Resetting all Alphas and Betas
        """
        A_vec = []
        B_vec = []
        for i in range(3):
            beta  = np.random.randint(1, 9) # beta between 1 and 8
            alpha = np.random.randint(beta+1, 10) # alpha between beta+1 and 9
            A_vec.append(0.1*alpha)
            B_vec.append(0.1*beta)
        self.A1, self.A2, self.A3 = A_vec
        self.B1, self.B2, self.B3 = B_vec

    def reset_markers(self):
        self.H1 = 1.0
        self.H2 = 1.0
        self.H3 = 1.0
        self.marker1, = self.ax1.plot([1], self.H1, 'ro', zorder=2, clip_on=False)
        self.marker2, = self.ax2.plot([1], self.H2, 'ro', zorder=2, clip_on=False)
        self.marker3, = self.ax3.plot([1], self.H3, 'ro', zorder=2, clip_on=False)
        self.marker_hysteresis, = self.ax4.plot([1], [self.H1+self.H2+self.H3], 'ro', zorder=5, clip_on=False)
        self.hysteresis_curve, = self.ax4.plot(self.input_history, self.output_history, color='green')
    
    def update_hysteron_value(self, u0, u1, alpha, beta, previous_state):
        """
        This is the core of the algorithm
        The value of the hysteron can only change if there is a threshold crossing.
        If the input increased (u1-u0>0), the threshold is alpha, otherwise beta.
        
        Case (a): input increased
        If the input is to the right of alpha, then the histeron is surely on, no matter what was its previous state
        Case (b): input decreased
        If the input is to the left of beta, then the histeron is surely off, no matter what was its previous state
        Is case we are neither to the right of alpha nor to the left of beta, then the hysteron remain as it was.
        """
        if (u1 - u0) > 0:
            threshold = alpha
            if u1 >= threshold:
                return 1.0
        else:
            threshold = beta
            if u1 <= threshold:
                return 0.0
        return previous_state
    def RegularPreisach(self, u):
        alpha = self.alpha_grid
        beta = self.beta_grid
        preisach_triangle = self.preisach_triangle
        # compare new input to previous input value and change hysteron values accordingly
        """This if u.new==0 is in order to cancel the last thin black patch in the left side
         of the priesach trianle and make it a whole gray when u=0"""
        if u.new==0:
                preisach_triangle=np.where(self.alpha_grid>self.beta_grid, 0, np.nan)
                preisach_triangle[-1][0]=1
        elif u.new > u.old: # if input increases  
            preisach_triangle = np.where(u.new>alpha, 1, preisach_triangle)
        elif u.new < u.old: # if input increases
            preisach_triangle = np.where(u.new<beta, 0, preisach_triangle)
            

        # values outside the presiach half-plane are set to nan
        preisach_triangle = np.where(alpha>=beta, preisach_triangle, np.nan)
        return preisach_triangle
    def xy_on(self, triangle):
        m = triangle 
        d = np.diff(m,axis=0)  # Calculate the difference between an element and the above
        x = np.where(d==1)[1]  # Getting all the x-columns coordinate
        y = self.hys_per_side-np.where(d==1)[0]-1  # Getting all the y-raws coordinate
        x = np.append(x, x[-1]+1)  # adding missing point at...
        y = np.append(y, y[-1])    # the end of the array
        step_index = np.where(np.diff(y)!= 0)[0]
        x = np.insert(x, step_index+1, x[step_index+1])
        y = np.insert(y, step_index, y[step_index])
        return [x/(self.hys_per_side-1), y/(self.hys_per_side-1)]
    
    def update_app(self, u):
        self.preisach_triangle = self.RegularPreisach(u)
        
        #update hysterons
        self.H1 = self.update_hysteron_value(u.old, u.new, self.A1, self.B1, self.H1)
        self.H2 = self.update_hysteron_value(u.old, u.new, self.A2, self.B2, self.H2)
        self.H3 = self.update_hysteron_value(u.old, u.new, self.A3, self.B3, self.H3)
        self.marker1.set_data([u.new], [self.H1])
        self.marker2.set_data([u.new], [self.H2])
        self.marker3.set_data([u.new], [self.H3])
        # update hysteresis graph
        self.input_history.append(u.new)
        self.output_history.append(self.H1 + self.H2 + self.H3)
        self.hysteresis_curve.set_data(self.input_history, self.output_history)
        self.marker_hysteresis.set_data([self.input_history[-1]], [self.output_history[-1]])
        self.on_state.remove()
        x_on, y_on = self.xy_on(self.preisach_triangle)
        self.on_state = self.ax5.fill_between(x_on, x_on, y_on, color="black")
my_app = app()
widgets.HBox([my_app.reset_button,my_app.input_u])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# The $\alpha$$\beta$ plane and "infinite" relays with different weights


Now we have much more relays, each of those has different $\alpha$  and $\beta$  thresholds and it's scaled by different weight $\mu$ . The relay hysterons can represent hysteresis with good accuracy now because of their amount.

In [3]:
class app_test(object):
    def __init__(self):
        #Setting all the data variables as instance variables here and in the on_button_clicked function 
        self.out = widgets.Output()
        self.hys_per_side = 101
        self.beta_values = np.around(np.linspace(0, 1, self.hys_per_side),2) # beta values
        self.alpha_values = np.around(np.linspace(1,0, self.hys_per_side),2) # alpha values
        self.beta_grid, self.alpha_grid = np.meshgrid(self.beta_values, self.alpha_values)
        #Dropdowns:
        self.dropdown1 = widgets.Dropdown(value='top_heavy', 
        options=['none','uniform', 'linear', 'top_heavy', 'right_heavy','left_heavy','center_light_alpha','center_light_beta',
             'center_heavy_alpha','single_line','upper_left'],
                                         description ="",layout = Layout(width='200px')
                                        )
        self.dropdown1.observe(self.on_button_clicked)
        self.dropdown2 = widgets.Dropdown(value='none', 
        options=['none','uniform', 'linear', 'top_heavy', 'right_heavy','left_heavy','center_light_alpha','center_light_beta',
             'center_heavy_alpha','single_line','upper_left'],
                                         description ="",layout = Layout(width='200px')
                                      ) 
        self.dropdown2.observe(self.on_button_clicked)
        #Slider
        self.sinputs = widgets.FloatSlider(
                            value=0, 
                            min=0, max=1, step=0.01,
                            description='input $u$',
                            continuous_update=True,
                            layout=Layout(width='50%')
                       )
        self.sinputs.observe(self.update_app, 'value')
        
        #Reset Button
        self.reset_button = widgets.Button(
                                description='Reset / Start',
                                icon="trash",
                                style={'font_weight': 'bold', 'button_color': 'yellow'}
                            )
        
        self.reset_button.on_click(self.on_button_clicked)# if you click it will activate the function
        self.on_button_clicked(1) # We force a click to reset all the plots

    def center_heavy_alpha_beta(self, m):
        mu = np.where(self.alpha_grid>=self.beta_grid, np.abs(0.5-m), np.nan)
        mu = np.where(self.alpha_grid>=self.beta_grid, np.abs(mu-1), np.nan)
        return mu
    def weights(self, method):
        # -----INPUTS-----Its the same as the original  
        alpha_grid = self.alpha_grid
        beta_grid = self.beta_grid
        if method=="none":
            return np.array([[0],[0]])
        mu = np.zeros((self.hys_per_side, self.hys_per_side))#creating empty array
        mu =  {
               'uniform': np.where(alpha_grid>=beta_grid, 1, np.nan),
               'linear': np.where(alpha_grid==beta_grid, 1, np.nan),
               'top_heavy': np.where(alpha_grid>=beta_grid, alpha_grid, np.nan),
               'bottom_heavy': np.where(alpha_grid>=beta_grid, 1-alpha_grid, np.nan),
               'right_heavy': np.where(alpha_grid>=beta_grid, beta_grid, np.nan),
               'left_heavy': np.where(alpha_grid>=beta_grid, 1-beta_grid, np.nan),
               'center_light_alpha': np.where(alpha_grid>=beta_grid, np.abs(0.5-alpha_grid), np.nan),
               'center_light_beta': np.where(alpha_grid>=beta_grid, np.abs(0.5-beta_grid), np.nan),
               'center_heavy_alpha': self.center_heavy_alpha_beta(alpha_grid),
               'center_heavy_beta': self.center_heavy_alpha_beta(beta_grid),
               'single_line': np.where(np.logical_and(0.3<beta_grid, beta_grid<0.5), 1, 0),
               'upper_left': np.where(np.logical_and(0.6>beta_grid, alpha_grid>0.95), 1, 0)
        }[method] #Inserting the whole thing into a dict and calling the right function as key
        mu = np.where(alpha_grid>=beta_grid, mu, np.nan)# Getting rid of the other triangle
        return mu / np.nansum(mu)

    def RegularPreisach(self, u, mu, outputs):
        """Simulation of discrete scalar Preisach model of hysteresis with single input
        determined by slider
        -----INPUTS-----
         u -- 1d array of previous input values
         alpha/beta -- 2d array of alpha/beta coordinates of hysterons
         mu -- 2d array with weights of hysterons
         preisach_triangle -- 2d array showing whether a given hysteron is on or off
         outputs -- 1d array with output values
         -----OUTPUTS-----
          preisach_triangle -- 2d array showing whether a given hysteron is on or off
          outputs -- 1d array with output values
         """
        alpha = self.alpha_grid
        beta = self.beta_grid
        preisach_triangle = self.preisach_triangle
        # compare new input to previous input value and change hysteron values accordingly
        """This if u.new==0 is in order to cancel the last thin black patch in the left side
         of the priesach trianle and make it a whole gray when u=0"""
        if u.new==0:
                preisach_triangle=np.where(self.alpha_grid>self.beta_grid, 0, np.nan)
                preisach_triangle[-1][0]=1
        elif u.new > u.old: # if input increases  
            preisach_triangle = np.where(u.new>alpha, 1, preisach_triangle)
        elif u.new < u.old: # if input increases
            preisach_triangle = np.where(u.new<beta, 0, preisach_triangle)
            

        # values outside the presiach half-plane are set to nan
        preisach_triangle = np.where(alpha>=beta, preisach_triangle, np.nan)
        # calculate weighted presiach triangle
        weighted_preisach = preisach_triangle*mu
        # new output value
        f = np.nansum(weighted_preisach)
        outputs = np.concatenate((outputs, np.array([f])))
        return outputs, preisach_triangle
    def draw_arrow(self,ax, start, end):
        ax.annotate('', xy=end, xytext=start, xycoords='data', textcoords='data',
                      arrowprops=dict(headwidth=4.0, headlength=4.0, width=0.2,
                                      facecolor = "black", linewidth = 0.5),zorder=0)
    
    def on_button_clicked(self, b):
        """The all_info array holds all the lines info by:
        
                 0                                1
        
        0   dropdown1 value       dict of line 1 - {"mu": mu1 ,
                                                    "outputs" : outputs1,
                                                    "plot" : plot 1 }


        1   dropdown2 value       dict of line 2 - {"mu": mu2 ,
                                                    "outputs" : outputs2,
                                                    "plot" : plot 2 }
        
        
        That means row 0 is for line 1 and row 1 is for line 2.
        -----------------------------------------------------------------------------------------------
        1) For example if we want to get mu1 -
        
           all_info[0][1]["mu"]
                    ^
                  line 1
                  
        
        2) If we want to get outputs2 - 
        
           all_info[1][1]["outputs"]
                    ^
                  line 2
        
        The column will always be 1 unless we want to use the dropdown value
        """
        self.preisach_triangle = np.where(self.alpha_grid>self.beta_grid, 0, np.nan)
        dict1 ={}
        dict2 ={}
        self.all_info = np.array([[str(self.dropdown1.value),dict1],[str(self.dropdown2.value),dict2]])
        self.hys_per_side=101
        self.all_info[0][1].update({'mu' : self.weights(str(self.dropdown1.value))})
        self.all_info[1][1].update({'mu' : self.weights(str(self.dropdown2.value))})
        self.all_info[0][1].update({'outputs' :  np.array([np.nansum(self.preisach_triangle)])})
        self.all_info[1][1].update({'outputs' :  np.array([np.nansum(self.preisach_triangle)])})
        self.initial_input = 0
        self.inputs = np.array([self.initial_input])
        
    
    #left=0.50, right=0.95,
     #left=0.10, right=0.35,   

        # Resetting plots
        plt.clf()
        self.fig = plt.figure(2, figsize=(10,5))
        gs1 = GridSpec(1, 1, width_ratios=[1],
                             height_ratios=[1],
                             left=0.50, right=0.95,
                             bottom=0.05, top=0.9,
                             wspace=0, hspace=0)
        gs2 = GridSpec(1, 1, width_ratios=[1],
                             height_ratios=[1],
                             left=0.05, right=0.35,  
                             bottom=0.50, top=0.9,
                             wspace=0, hspace=0)
        gs3 = GridSpec(1, 2, width_ratios=[1,1],
                             height_ratios=[1],
                             left=-0, right=0.35,  
                             bottom=0.05, top=0.45,
                             wspace=0.55, hspace=0.5)
        self.ax1 = plt.subplot(gs1[0, 0])
        self.ax2 = plt.subplot(gs2[0, 0])
        self.ax2.set_aspect('equal', adjustable='box')
        self.ax3 = plt.subplot(gs3[0, 0])
        self.ax4 = plt.subplot(gs3[0, 1])
        # Main graph
        plt.subplots_adjust(bottom = 0.0,hspace = 1) 
        self.ax1.set_xlim([-0.05, 1.05])
        self.ax1.set_ylim([-0.05, 1.05])
        self.ax1.axis('off')
        self.ax1.text(-0.1, 1.12, "Output") 
        self.ax1.text(1.1, -0.05, r"$u$")   
        self.draw_arrow(self.ax1, (-0.05, -0.05),(-0.05, 1.05))
        self.draw_arrow(self.ax1, (-0.05, -0.05),(1.05, -0.05))
        self.all_info[0][1].update({'plot' : self.ax1.plot(self.inputs[0] , self.all_info[0][1]["outputs"][0],
                                                    color="tab:red",label="Weight 1")[0]})
        self.all_info[0][1].update({'marker' : self.ax1.plot(-1 ,-1
                                                   ,marker='o',
                                                    color="tab:red")[0]})
        self.all_info[1][1].update({'plot' : self.ax1.plot(self.inputs[0] , self.all_info[1][1]["outputs"][0],
                                                    color="tab:blue",label="Weight 2")[0]})
        self.all_info[1][1].update({'marker' : self.ax1.plot(-1 ,-1
                                                   ,marker='o',
                                                    color="tab:blue")[0]})
        self.ax1.legend(loc="upper left")
        
        # The two weight triangles
        self.my_Reds = self.truncate_colormap(plt.get_cmap('Reds'), 0.2, 1.0)
        self.ax3.imshow(np.fliplr(np.flip(self.all_info[0][1]["mu"])), cmap=self.my_Reds,
                        vmin=0-0*np.nanmax(self.all_info[0][1]["mu"]),vmax=np.nanmax(self.all_info[0][1]["mu"]) ) 

        self.ax3.set_xlim([-0.05, 100.05])
        self.ax3.set_ylim([-0.05, 100.05])
        self.ax3.set_xticks([])
        self.ax3.set_yticks([])
        self.ax3.spines['top'].set_visible(False)
        self.ax3.spines['right'].set_visible(False)
        self.ax3.spines['bottom'].set_visible(False)
        self.ax3.spines['left'].set_visible(False)
        self.ax3.set_xlabel('\n Weight 1     ')

        self.my_Blues = self.truncate_colormap(plt.get_cmap('Blues'), 0.2, 1.0)
        self.ax4.imshow(np.fliplr(np.flip(self.all_info[1][1]["mu"])), cmap=self.my_Blues,
                        vmin=0-0*np.nanmax(self.all_info[1][1]["mu"]),vmax=np.nanmax(self.all_info[1][1]["mu"]) ) 

        self.ax4.set_xlim([-0.05, 100.05])
        self.ax4.set_ylim([-0.05, 100.05])
        self.ax4.set_xticks([])
        self.ax4.set_yticks([])
        self.ax4.spines['top'].set_visible(False)
        self.ax4.spines['right'].set_visible(False)
        self.ax4.spines['bottom'].set_visible(False)
        self.ax4.spines['left'].set_visible(False)
        self.ax4.set_xlabel('\n Weight 2            ')
        
        # The interactive preisach triangles    
        self.ax2.set_xlim([-0.003,1])
        self.ax2.set_ylim([0,1])
        self.ax2.axis('off')
        self.ax2.set_title("Limit Triangle\n")
        self.ax2.text(1.01, 0, "\u03B2") 
        self.ax2.text(-0.02, 1.09, "\u03B1") 
        self.ax2.annotate('', xy=(0, 0), xycoords=('data'),
                          xytext=(0, 1.075), textcoords='data',
                          ha='left', va='center',
                          arrowprops=dict(arrowstyle='<|-', fc='black'),zorder=2)
        self.draw_arrow(self.ax2, (0,0),(0.983, 0))
        self.x = np.linspace(0, 1, self.hys_per_side)
        self.ax2.fill_between(self.x, self.x, 1, color="gray")
        self.on_state = self.ax2.fill_between([0], [0], 0, color="black")
        
        # After ploting all plots we can get rid of the none line 
        self.all_info = np.delete(self.all_info, np.where(self.all_info == "none")[0], axis=0)
        
        # Resseting slider value - MUST BE in the end of function       
        """
        Everytime we activated reset, the function xy_on would be activate and get preisch triangle as a whole 0 triangle
        and thus making errors. That why we update the triangle with a single 1 in the bottom after we displayed it as a
        whole 0. It excludes the function xy_on making errors by: 
        
        def xy_on(self, triangle):
            m = triangle 
            d = np.diff(m,axis=0) <---- never be empty
            x = np.where(d==1)[1] <---- never be empty
            x = np.append(x, x[-1]+1)
                                ^
                                never try to search last element in an empty array
            ...
        """
        self.preisach_triangle[-1][0]=1 
        
        
        # This part is active every set up with the value of 1. 
        self.sinputs.value=0 # First we refer the u as zero in order to make a change to 1, no matter what value had been before.
        self.sinputs.observe(self.update_app, 'value')
        self.sinputs.value=1
        self.sinputs.observe(self.update_app, 'value')
        self.inputs = [1]
        for line in self.all_info:
            line=line[1] # Using only the dict of each line
            y = line["plot"].get_ydata()[-1]
            line["plot"].set_ydata([])
            line["plot"].set_xdata([])
            line["outputs"]=[y]
        
        
        
    def xy_on(self, triangle):
        m = triangle 
        d = np.diff(m,axis=0)  # Calculate the difference between an element and the above
        x = np.where(d==1)[1]  # Getting all the x-columns coordinate
        y = self.hys_per_side-np.where(d==1)[0]-1  # Getting all the y-raws coordinate
        x = np.append(x, x[-1]+1)  # adding missing point at...
        y = np.append(y, y[-1])    # the end of the array
        step_index = np.where(np.diff(y)!= 0)[0]
        x = np.insert(x, step_index+1, x[step_index+1])
        y = np.insert(y, step_index, y[step_index])
        return [x/(self.hys_per_side-1), y/(self.hys_per_side-1)]
    
    def update_app(self, u):
        self.inputs = np.concatenate((self.inputs, np.array([u.new])))
        for line in self.all_info:
            line=line[1] # Using only the dict of each line
            line["outputs"], self.preisach_triangle = self.RegularPreisach(u, line["mu"] , line["outputs"])
            line["plot"].set_ydata(line["outputs"])
            line["plot"].set_xdata(self.inputs)
            line["marker"].set_ydata(line["outputs"][-1])
            line["marker"].set_xdata(self.inputs[-1])
        self.on_state.remove()
        x_on, y_on = self.xy_on(self.preisach_triangle)
        self.on_state = self.ax2.fill_between(x_on, x_on, y_on, color="black")
    
    def truncate_colormap(self, cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

my_app_test = app_test() 
widgets.HBox([my_app_test.dropdown1, my_app_test.dropdown2, my_app_test.sinputs,my_app_test.reset_button])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …